In [ ]:
!python -mpip install --quiet fitparse==1.1.0

In [ ]:
from fitparse import FitFile


def to_deg(semicircles):
    return semicircles * (180 / 2 ** 31)


fitfile = FitFile("5006234923.fit")
positions = [
    (
        to_deg(record.get_value("position_lat")),
        to_deg(record.get_value("position_long")),
    )
    for record in fitfile.get_messages("record")
]
session = next(fitfile.get_messages(name="session"))
timestamp = session.get_value("timestamp")
start_position = (
    to_deg(session.get_value("start_position_lat")),
    to_deg(session.get_value("start_position_long")),
)
total_distance = session.get_value("total_distance")

print("timestamp", timestamp)
print("start_position", start_position)
print("total_distance", total_distance)

In [ ]:
from statistics import mean

lats, longs = zip(*positions)
mean_position = (mean(lats), mean(longs))

In [ ]:
import folium

m = folium.Map(mean_position, zoom_start=14)
for position in positions:
    folium.Circle(position, radius=1).add_to(m)
m

In [ ]:
import altair as alt
import pandas as pd

data = pd.DataFrame(
    (
        (record.get_value("timestamp"), record.get_value("speed"))
        for record in fitfile.get_messages("record")
    ),
    columns=["timestamp", "speed"],
)
alt.Chart(data).mark_line().encode(
    x="timestamp", y="speed",
)